In [ ]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the folder contianing the subfolders of images.
# Function inputs args: test [bool] --> When 1, will change the gui title to that of the test gui. 
# Function output 1: The path of the folder selected by the user. 
def folder_selection_dialog(test = 0):
    root = Tk()
    if test:
        root.title('Please select the "test" folder within the downloaded package.')
        root.filename = filedialog.askdirectory(initialdir="/", title="Please select the 'test' folder within the downloaded package.")
    else: 
        root.title('Please select the directory containing the subfolders of image data')
        root.filename = filedialog.askdirectory(initialdir="/", title="Please select the directory containing the subfolders of image data")
    parent_directory = root.filename
    root.destroy()

    return parent_directory

In [ ]:
import os as os

# A function to create a list of image paths, such that these images can later be used to make a movie. 
# Function input args: parent_directory [string] --> The directory containing the subfolders of images. 
# Function output arg 1: image_paths [list] --> A list of the image paths, where each path is a string. 
def list_image_paths(parent_directory):
    
    # First, list the subfolders in the parent directory.
    subfolders = [_ for _ in os.listdir(parent_directory)]

    # Iterate through the list of subfolders and extract the images contained within them. 
    image_paths = []
    for i in range(len(subfolders)):
        
        current_subfolder = subfolders[i]
        subfolder_dir = os.path.join(parent_directory, current_subfolder)
        images_in_subfolder = [os.path.join(subfolder_dir, image) for image in os.listdir(subfolder_dir) if image.endswith('.png')]
        image_paths.append(images_in_subfolder)
        
    image_paths = [image for sublist in image_paths for image in sublist]
    
    return image_paths

In [ ]:
# A function to take the list of image paths, load in said images, and convert them to a movie. 
# Function input arg 1:
# Function input arg 2: 
def create_movie():
    